In [1]:
import pandas as pd
import numpy as np

# Load Dataset


In [2]:
def read_file(path):
    payload = []
    with open(path,'r',encoding='utf8') as f:
        for line in f:
            payload.append(line[:-1]) #remove '\n'
    return np.array(payload)

### Bad queries payloads

In [3]:
df_badQueries = pd.DataFrame(columns = ['payload','is_malicious'])
df_badQueries['payload'] = read_file('./badqueries.txt')
df_badQueries['is_malicious'] = 1 

In [4]:
df_badQueries.tail(10)

,payload,is_malicious
48116,<script>eval(location.hash.slice(1)</script>,1
48117,"<base href=data:/,-alert(1)/>",1
48118,<svg><set href=#script attributeName=href to=d...,1
48119,"“/>.<<img src=x onerror=alert(1)//\""&gt;>&lt;&gt",1
48120,“>><<img src=x onerror=alert(1);//>>;,1
48121,"""text </script><script>alert(1)</script>""",1
48122,<marquee loop=1 width=0 onfinish=1/confirm`/1/...,1
48123,<marquee loop=1 width=0 onfinish=confirm(1)>0<...,1
48124,<abeon style=font-size:12px onmouseover=confir...,1
48125,<svg onload=location='//p0.al'>,1


### Good queries payloads

In [5]:
df_goodQueries = pd.DataFrame(columns = ['payload','is_malicious'])
df_goodQueries['payload'] = read_file('./goodqueries.txt')
df_goodQueries['is_malicious'] = 0

In [6]:
df_goodQueries.tail(10)

,payload,is_malicious
1294521,/marrying_gifts/,0
1294522,/mir_download/,0
1294523,/20050308145407/,0
1294524,/javascript/migrated.html,0
1294525,/ms06-054/,0
1294526,/197573/,0
1294527,/javascript/management.php,0
1294528,/javascript/minute.csproj,0
1294529,/tex-lg/,0
1294530,/javascript/42.tar.gz,0


In [7]:
# Total amount of entries
len(df_badQueries) + len(df_goodQueries)

1342657

In [24]:
df_badQueries.to_csv('badQueriesCSV.csv', sep=',', encoding='utf-8')
df_goodQueries.to_csv('goodQueriesCSV.csv', sep=',', encoding='utf-8')

# Creating s3 bucket using boto3
    Bucket name: queries-dataset

In [8]:
import boto3
from botocore.exceptions import ClientError

BUCKET_NAME = 'queries-dataset'
S3_RESOURCE = 's3'

In [9]:

def create_bucket(bucket_name):
    # Create an S3 bucket, the bucket is created in the S3 default region. 
    try:
            s3_client = boto3.client(S3_RESOURCE)
            s3_client.create_bucket(Bucket=bucket_name)
    except ClientError as e:
        print(e)
        print(f'{bucket_name} wasn\'t created')
        return
    print(f'{bucket_name} was created')
    

def show_existing_buckets():
    # Retrieve the list of existing buckets check the bucket name queries-dataset is exists
    s3 = boto3.client(S3_RESOURCE)
    response = s3.list_buckets()

    # Output the bucket names
    print('Existing buckets:')
    for bucket in response['Buckets']:
        print(f'{bucket["Name"]}')
    

In [10]:
create_bucket(BUCKET_NAME)
show_existing_buckets()

queries-dataset was created
Existing buckets:
  queries-dataset


# Upload Dataset

In [13]:
def upload_file(file_name, bucket, object_name=None):

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    s3_client = boto3.client(S3_RESOURCE)
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        print(e)
        print(f'{file_name} failed to upload object')
        return
    
    print(f'{file_name} uploaded succesfully')


In [17]:
upload_file('badqueries.txt',BUCKET_NAME)
upload_file('goodqueries.txt',BUCKET_NAME)

badqueries.txt uploaded succesfully
goodqueries.txt uploaded succesfully


In [25]:
upload_file('badQueriesCSV.csv',BUCKET_NAME)
upload_file('goodQueriesCSV.csv',BUCKET_NAME)

badQueriesCSV.csv uploaded succesfully
goodQueriesCSV.csv uploaded succesfully
